In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [10]:
spark = SparkSession\
    .builder\
    .appName("analisisLogs")\
    .getOrCreate()

print("Активные Spark сессии:", spark.sparkContext.uiWebUrl)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/19 17:55:36 WARN Utils: Your hostname, king, resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface enp4s0)
25/07/19 17:55:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/19 17:55:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/19 17:55:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/07/19 17:55:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/07/19 17:55:47 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


Активные Spark сессии: http://192.168.0.105:4043


In [11]:
!ls

analisis_info.ipynb  web_server_logs.csv


In [12]:
PATH = "web_server_logs.csv"

In [13]:
spark.read.csv(PATH).show(5)

+--------------+--------------------+------+--------------------+-------------+-------------+
|           _c0|                 _c1|   _c2|                 _c3|          _c4|          _c5|
+--------------+--------------------+------+--------------------+-------------+-------------+
|            ip|           timestamp|method|                 url|response_code|response_size|
|107.234.133.87|2025-05-12T16:19:...|   PUT|category/wp-conte...|          200|         5678|
|  215.25.19.75|2025-01-15T07:34:...|   PUT|           list/blog|          500|         8332|
|122.130.84.128|2025-07-01T02:00:...|  POST|           posts/tag|          301|         1687|
|  174.123.16.3|2025-03-23T19:27:...|   PUT|                tags|          404|         3539|
+--------------+--------------------+------+--------------------+-------------+-------------+
only showing top 5 rows


In [16]:
df = spark.read.csv(PATH, header=True)

df.show(2, False, True)

-RECORD 0-----------------------------------
 ip            | 107.234.133.87             
 timestamp     | 2025-05-12T16:19:50.875140 
 method        | PUT                        
 url           | category/wp-content/tags   
 response_code | 200                        
 response_size | 5678                       
-RECORD 1-----------------------------------
 ip            | 215.25.19.75               
 timestamp     | 2025-01-15T07:34:44.835729 
 method        | PUT                        
 url           | list/blog                  
 response_code | 500                        
 response_size | 8332                       
only showing top 2 rows


In [17]:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: string (nullable = true)
 |-- response_size: string (nullable = true)



In [24]:
df = df.withColumn("timestamp", F.col("timestamp").cast("date"))

In [25]:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: date (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: string (nullable = true)
 |-- response_size: string (nullable = true)



In [26]:
df.show(3)

+--------------+----------+------+--------------------+-------------+-------------+
|            ip| timestamp|method|                 url|response_code|response_size|
+--------------+----------+------+--------------------+-------------+-------------+
|107.234.133.87|2025-05-12|   PUT|category/wp-conte...|          200|         5678|
|  215.25.19.75|2025-01-15|   PUT|           list/blog|          500|         8332|
|122.130.84.128|2025-07-01|  POST|           posts/tag|          301|         1687|
+--------------+----------+------+--------------------+-------------+-------------+
only showing top 3 rows


<h2>Групировка по ip и вывод самых активных</h2>

In [30]:
df\
    .groupBy("ip")\
    .agg(F.count("*").alias("request_count"))\
    .orderBy(F.col("request_count").desc())\
    .limit(10)\
    .show()

+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|91.233.187.225|            2|
| 84.238.175.42|            1|
|75.108.173.172|            1|
|104.156.144.63|            1|
| 130.61.86.206|            1|
|45.112.254.216|            1|
|50.220.165.111|            1|
|   70.46.74.58|            1|
|  164.71.54.76|            1|
| 150.154.24.35|            1|
+--------------+-------------+



<h2>Групировка по HTTP и подсчет количества</h2>

In [31]:
df\
    .groupBy("method")\
    .agg(F.count("*").alias("request_count"))\
    .orderBy(F.col("request_count").desc())\
    .show()

+------+-------------+
|method|request_count|
+------+-------------+
|  POST|        25153|
|   GET|        25146|
|   PUT|        24885|
|DELETE|        24816|
+------+-------------+



<h2>Вывод записей с ошибкой 404</h2>

In [39]:
df.filter(df["response_code"] == 404).show(5)

+--------------+----------+------+--------------------+-------------+-------------+
|            ip| timestamp|method|                 url|response_code|response_size|
+--------------+----------+------+--------------------+-------------+-------------+
|  174.123.16.3|2025-03-23|   PUT|                tags|          404|         3539|
| 43.55.103.134|2025-02-08|   GET|wp-content/wp-con...|          404|         7352|
| 4.123.174.239|2025-02-14|DELETE|wp-content/app/wp...|          404|          270|
|  34.23.138.28|2025-07-17|   GET|    categories/posts|          404|         8403|
|171.217.108.43|2025-03-28|DELETE|categories/search...|          404|         8083|
+--------------+----------+------+--------------------+-------------+-------------+
only showing top 5 rows


In [40]:
df.where(df.response_code == 404).show(5)

+--------------+----------+------+--------------------+-------------+-------------+
|            ip| timestamp|method|                 url|response_code|response_size|
+--------------+----------+------+--------------------+-------------+-------------+
|  174.123.16.3|2025-03-23|   PUT|                tags|          404|         3539|
| 43.55.103.134|2025-02-08|   GET|wp-content/wp-con...|          404|         7352|
| 4.123.174.239|2025-02-14|DELETE|wp-content/app/wp...|          404|          270|
|  34.23.138.28|2025-07-17|   GET|    categories/posts|          404|         8403|
|171.217.108.43|2025-03-28|DELETE|categories/search...|          404|         8083|
+--------------+----------+------+--------------------+-------------+-------------+
only showing top 5 rows


In [45]:
df\
    .groupBy("timestamp")\
    .agg(F.sum("response_size").alias("total_response_size"))\
    .orderBy(F.col("timestamp"))\
    .show()

+----------+-------------------+
| timestamp|total_response_size|
+----------+-------------------+
|2025-01-01|          2449098.0|
|2025-01-02|          2510894.0|
|2025-01-03|          2584497.0|
|2025-01-04|          2540923.0|
|2025-01-05|          2457712.0|
|2025-01-06|          2506474.0|
|2025-01-07|          2573893.0|
|2025-01-08|          2538078.0|
|2025-01-09|          2766398.0|
|2025-01-10|          2766685.0|
|2025-01-11|          2487781.0|
|2025-01-12|          2762943.0|
|2025-01-13|          2333733.0|
|2025-01-14|          2513003.0|
|2025-01-15|          2564854.0|
|2025-01-16|          2334054.0|
|2025-01-17|          2207826.0|
|2025-01-18|          2310628.0|
|2025-01-19|          2594186.0|
|2025-01-20|          2421061.0|
+----------+-------------------+
only showing top 20 rows
